In [1]:
import pandas as pd
import dask.dataframe as dd
import os
import sys
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
path_stocks = '/home/rishabh/Documents/Data_Science_Projects/DS_Medium_post/stocks/Stocks'

In [3]:
# os.listdir(path_stocks)

In [91]:
%time
df = pd.read_csv('/home/rishabh/Documents/Data_Science_Projects/DS_Medium_post/stocks/Stocks/cpl.us.txt')
cols = df.columns

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 11 µs


In [96]:
cols

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt'], dtype='object')

In [98]:
# df_list = []
df_Open = pd.DataFrame(list())
df_High = pd.DataFrame(list())
df_Low = pd.DataFrame(list())
df_Close = pd.DataFrame(list())
df_Volume = pd.DataFrame(list())
df_OpenInt = pd.DataFrame(list())

for stock in os.listdir(path_stocks):
    filesize = os.path.getsize(path_stocks+'/'+stock)

    if filesize == 0:continue
    if stock.split('.')[2] != 'txt':continue
    path = path_stocks+'/'+stock
    for col in cols:
        
        if col == 'Date':continue
        df_temp = pd.read_csv(path, usecols=['Date', col], index_col='Date')
        new_col = stock.split('.')[0]+col
        if col == 'Open':
            df_Open[new_col] = df_temp[col]
            
        elif col == 'High':
            df_High[new_col] = df_temp[col]
            
        elif col == 'Low':
            df_Low[new_col] = df_temp[col]
            
        elif col == 'Close':
            df_Close[new_col] = df_temp[col]
            
        elif col == 'Volume':
            df_Volume[new_col] = df_temp[col]
            
        elif col == 'OpenInt':
            df_OpenInt[new_col] = df_temp[col]
            
#         sys.exit(0)
        del(df_temp)



In [99]:
df_Open.index = pd.to_datetime(df_Open.index)
df_High.index = pd.to_datetime(df_High.index)
df_Low.index = pd.to_datetime(df_Low.index)
df_Close.index = pd.to_datetime(df_Close.index)
df_Volume.index = pd.to_datetime(df_Volume.index)
df_OpenInt.index = pd.to_datetime(df_OpenInt.index)

In [100]:
df_Open.to_csv('df_Open.csv')
df_High.to_csv('df_High.csv')
df_Low.to_csv('df_Low.csv')
df_Close.to_csv('df_Close.csv')
df_Volume.to_csv('df_Volume.csv')
df_OpenInt.to_csv('df_OpenInt.csv')


In [102]:
pd.read_csv('df_Open.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3201 entries, 0 to 3200
Columns: 7164 entries, Date to discbOpen
dtypes: float64(7163), object(1)
memory usage: 175.0+ MB


In [88]:
from bokeh.plotting import figure, output_file, show

def plot_stock(stock,df):
    
    '''
    This function plots the time-series line plot of a stock
    
    input: stock name: ex: aapl, amzn, tsla
    
    output: line plot of the open price with 2sigma bands
    '''
    x = df.index.values.tolist()
    y0 = df[stock+'_open'].values.tolist()
    y0_rolling_mean= df[stock+'_open'].rolling(window=10).mean()
    y0_rolling_std= df[stock+'_open'].rolling(window=10).std()


    # output to static HTML file
    # output_file("log_lines.html")

    # create a new plot
    p = figure(
       tools="pan,box_zoom,reset,save",
        title="Open Price",
       x_axis_label='date',
        y_axis_label='Open Price',
        x_axis_type='datetime',
        plot_width=1000
    )


    p.line(x, y0, legend_label=stock, line_width=3)
    p.line(x, y0_rolling_mean+(2*y0_rolling_std), legend_label="aapl+roll", line_color="red")
    p.line(x, y0_rolling_mean-(2*y0_rolling_std), legend_label="aapl-roll", line_color="red")

    # show the results
    show(p)


In [89]:
plot_stock('aapl', df)